# **Entrega 1: Extract and Load**

#### Instalar Pandas y Request

Asegurarse de tener instaladas las librerías requests, pandas y deltalake. 

```bash
pip install -r requirements.txt
```

#### Importar librerías a utilizar

In [1]:
import requests
import pandas as pd
from deltalake import write_deltalake, DeltaTable
from datetime import datetime
from configparser import ConfigParser
import os
pd.options.display.max_rows = None # Visualizar todas las filas de los df
pd.options.display.max_columns = None # Visualizar todas las columnas de los df

#### Definir funciones y diccionario de ciudades a utilizar

- Para mayor seguridad, las credenciales de la API se obtienen desde un archivo 'pipeline.conf'.

In [2]:
# Función para obtener datos desde la API
def get_data(endpoint, cities):
    # Leer archivo de configuración
    config = ConfigParser()
    config.read('pipeline.conf')

    # Obtener valores desde el archivo de configuración
    url_base = config['URL']['url_base']
    api_key = config['API']['api_key']

    # Construir solicitudes para cada ciudad
    data_list = []
    for city in cities:
        url = f"{url_base}/{endpoint}?lat={city['lat']}&lon={city['lon']}&appid={api_key}&units=metric"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            data_list.append(data)
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener datos para {city['name']}: {e}")
            continue
    return data_list

# Función para construir el DataFrame a partir de los datos obtenidos
def build_table(data_list, x=None):
    try:
        df = pd.json_normalize(data_list, record_path=x)
        return df
    except ValueError:
        print("Los datos no están en el formato esperado")
        return None

# Lista de ciudades con sus coordenadas
cities = [
    {'name': 'Córdoba', 'lat': -31.4135, 'lon': -64.1811},
    {'name': 'Buenos Aires', 'lat': -34.6132, 'lon': -58.3772},
    {'name': 'Rosario', 'lat': -32.9442, 'lon': -60.6505}
]

------

### **1. Current Weather - Endpoint de Datos Dinámicos - Extracción Incremental**

API URL: https://openweathermap.org/current

Con intervalos de una hora, se extraen datos del tiempo meteorológico actual. Esta extracción-carga se asegura mediante validaciones de guardar incrementalmente solo aquellos datos con fecha y hora posteriores a la de los datos existentes.

In [22]:
# Endpoint para el clima actual
current_endpoint = 'weather'

# Llamar a la función get_data
current_data = get_data(current_endpoint, cities)

# Construir el DataFrame aplanando JSON
current_df = build_table(current_data)

# Eliminar estas columnas innecesarias si es que existen
# Se realiza esta transformación en esta etapa para mantener desde el inicio un schema acorde y así evitar inconsistencias ya que la API no suele devolver siempre la misma cantidad de columnas.
columns_to_drop = ['weather', 'base', 'dt', 'timezone', 'cod', 'clouds.all', 'sys.type',
                    'sys.id', 'sys.country', 'sys.sunrise', 'sys.sunset', 'wind.gust', 'rain.1h']                  
current_df = current_df.drop(columns=[col for col in columns_to_drop if col in current_df.columns], axis=1, errors='ignore')

# Agregar al dataframe columnas con fecha y hora actual
current_df['date'] = datetime.now().strftime('%Y-%m-%d')
current_df['hour_time'] = datetime.now().strftime('%H')

# Directorio de la capa bronze
bronze_dir_current = 'datalake/bronze/current_weather_data'

# Validar si el directorio bronze existe
if not os.path.exists(bronze_dir_current):
    write_deltalake(
        table_or_uri=bronze_dir_current,
        data=current_df,
        mode='append',
        partition_by=['date', 'hour_time']
    )
    print("Delta Lake (bronze) inicializado con los datos actuales.")
else:
    try:
        # Obtener los datos existentes del bronze Delta Lake
        existing_dt = DeltaTable(bronze_dir_current)
        existing_data = existing_dt.to_pandas()

        # Validar si ya existen datos para la misma combinación de date y hour_time
        date_to_check = current_df['date'].iloc[0]
        hour_to_check = current_df['hour_time'].iloc[0]

        if ((existing_data['date'] == date_to_check) & (existing_data['hour_time'] == hour_to_check)).any():
            print(f"Ya existen datos en bronze para la fecha {date_to_check} y la hora {hour_to_check}. La inserción ha sido cancelada.")
        else:
            # Si no hay duplicados, escribir los datos en el bronze Delta Lake
            write_deltalake(
                table_or_uri=bronze_dir_current,
                data=current_df,
                mode='append',  # Usa 'append' para realizar incrementos en cada extracción
                partition_by=['date', 'hour_time']  # Columnas de partición
            )
            print(f"Datos de la última hora ({hour_to_check}) agregados correctamente a la capa bronze.")
    except Exception as e:
        print(f"Error al cargar los datos en el bronze: {e}")

# Ver cantidad total de registros actuales de la bronze deltatable actualizada
if os.path.exists(bronze_dir_current):
    current_dt = DeltaTable(bronze_dir_current)
    print(f"Cantidad total de registros en la bronze deltatable actualizada: {current_dt.to_pandas().shape[0]}")
    
# Descomentar las siguientes lineas para imprimir el último dataframe extraído-cargado
# current_df

Datos de la última hora (20) agregados correctamente a la capa bronze.
Cantidad total de registros en la bronze deltatable actualizada: 27


-------

### **2. History Weather - Endpoint de Datos Estáticos - Extracción Full**

API URL: https://openweathermap.org/forecast5

La ejecución del siguiente código nos permitirá tener el pronótico de los próximos 5 días cada vez que ejecutemos el programa, sobreescribiendo los datos anteriores. La API arroja datos nuevos cada 3 horas.

In [21]:
# API 5 day weather forecast - Pronóstico de clima para los próximos 5 días (data c/3hs)
history_endpoint = 'forecast'

# Obtener datos solo para Córdoba
cordoba_data = [city for city in cities if city['name'] == 'Córdoba'] 

# Traer la data solo para Córdoba
history_data = get_data(history_endpoint, cordoba_data)

# Contruir tabla con datos json aplanados
history_df = build_table(history_data, 'list')

# URL directorio
bronze_dir_history = 'datalake/bronze/history_weather_data'

# Generar deltalake
write_deltalake(
    table_or_uri=bronze_dir_history,
    data=history_df,
    mode='overwrite',  # 'overwrite' para extracción full. Sobreescribe los datos antiguos
)

# Ver cantidad de registros en la tabla actualizada
history_dt = DeltaTable(bronze_dir_history)
print(f'Cantidad total de registros en la bronze deltatable actualizada: {history_dt.to_pandas().shape[0]}') # La cantidad se mantiene constante en 40 registros porque la data se sobreescribe

# Descomentar la sigueinte línea para ver el último DataFrame extraído
# history_df

Cantidad total de registros en la bronze deltatable actualizada: 40
